In [1]:
# 使用函式庫
from typing import List
import random
from IPython.display import display, Javascript
from ipywidgets import Button, Label, GridspecLayout, VBox, Layout, HBox, Text
import asyncio
import time

In [2]:
"""蛇的類別，負責蛇一開始的初始化與移動"""
class Snake:
    def __init__(self, player_id, k):
        """
        Attributes:
            player_id: 玩家編號
            body: 蛇的身體
            direction: 蛇的移動方向
            score: 蛇的分數
            food_eaten: 蛇有沒有吃到食物
        """
        self.player_id = player_id
        self.body = []
        self.direction = (0, 0)
        self.score = k-1
        self.food_eaten = 0

    def spawn(self):
        self.body = [(random.randint(7, 12), random.randint(7, 12))]
        k = self.score
        last_body = self.body[0]
        while k > 0:
            self.body.append((last_body[0]-1, last_body[1]))
            last_body = (last_body[0]-1, last_body[1])
            k -= 1
        self.direction = (1, 0)

    def move(self):

        head = self.body[0]
        new_head = (head[0] + self.direction[0], head[1] + self.direction[1])

        self.body.insert(0, new_head)
        if self.food_eaten > 0:
            self.food_eaten -= 1
        else:
            self.body.pop()

In [3]:
"""食物的類別，負責隨機產生食物位置"""
class Food:
    def __init__(self):
        """
        Attributes:
            position: 食物位置
        """
        self.position = (0, 0)
        self.spawn()

    def spawn(self):
        self.position = (random.randint(0, 19), random.randint(0, 19))

In [4]:
"""遊戲的類別，負責增加蛇、移除蛇、增加食物、移除食物、移動蛇的位置、檢查碰撞、更新地圖"""
class MultiplayerSnakeGame:
    def __init__(self):
        """
        Attributes:
            width: 場地寬度
            height: 場地高度
            snakes: 儲存蛇的陣列
            foods: 儲存食物的陣列
            map: 地圖
        """
        self.width = 20
        self.height = 20
        self.snakes: List[Snake] = []
        self.foods: List[Food] = []
        self.map = [["" for _ in range(self.width)] for _ in range(self.height)]

    def add_snake(self, snake: Snake):
        """
        新增蛇
        """
        self.snakes.append(snake)
        for x, y in snake.body:
            self.map[x][y] = snake.player_id

    def remove_snake(self, player_id: str):
        """
        移除蛇
        """
        self.snakes = [snake for snake in self.snakes if snake.player_id != player_id]
        for x in range(self.width):
            for y in range(self.height):
                if self.map[x][y] == player_id:
                    self.map[x][y] = ""

    def add_food(self, food: Food):
        """
        將新食物放在地圖上
        """
        self.foods.append(food)
        self.map[food.position[0]][food.position[1]] = "food"

    def remove_food(self, position):
        """
        將食物從地圖移除
        """
        self.foods = [food for food in self.foods if food.position != position]
        self.map[position[0]][position[1]] = ""

    def move_snakes(self):
        """
        移動蛇
        """
        for snake in self.snakes:
            snake.move()

    def change_direction(self, player_id, direction):
        """
        改變方向
        """
        print(f"Changing direction of {player_id} to {direction}")
        for snake in self.snakes:
            if snake.player_id == player_id:
                if (
                    snake.direction == (1, 0) and direction == (-1, 0)
                    or snake.direction == (-1, 0) and direction == (1, 0)
                    or snake.direction == (0, 1) and direction == (0, -1)
                    or snake.direction == (0, -1) and direction == (0, 1)
                ):
                    return
                snake.direction = direction

    def check_collisions(self):
        """
        檢查碰撞
        """
        for snake in self.snakes:
            head = snake.body[0]
            if (
                head[0] < 0
                or head[0] >= self.width
                or head[1] < 0
                or head[1] >= self.height
            ):
                self.remove_snake(snake.player_id)
            elif (
                self.map[head[0]][head[1]] != "" and self.map[head[0]][head[1]] != "food"
            ):
                print("Collision")
                self.remove_snake(snake.player_id)
            elif head in [food.position for food in self.foods]:
                snake.score += 1
                snake.food_eaten += 1
                self.remove_food(head)

    def update_map(self):
        """
        更新地圖
        """
        self.map = [["" for _ in range(self.width)] for _ in range(self.height)]
        for snake in self.snakes:
            for x, y in snake.body:
                self.map[x][y] = snake.player_id
        for food in self.foods:
            self.map[food.position[0]][food.position[1]] = "food"

    def spawn_food(self):
        """
        產生新食物
        """
        while len(self.foods) < 10:
            new_food = Food()
            new_food.spawn()
            while self.map[new_food.position[0]][new_food.position[1]] != "":
                new_food.spawn()
            self.add_food(new_food)

    def spawn_snake(self, player_id, k):
        """
        產生新蛇
        """
        new_snake = Snake(player_id, k)
        new_snake.spawn()
        self.add_snake(new_snake)

    def update(self):
        """
        更新：
        移動蛇
        檢查碰撞
        更新地圖
        產生食物
        """
        self.move_snakes()
        self.check_collisions()
        self.update_map()
        self.spawn_food()

In [5]:
#產生新的遊戲 game 並設定參數
game = MultiplayerSnakeGame()
game_active = False
FRAME_RATE = 0.5

#產生場地的函數
def create_grid(data):
    rows = len(data)
    cols = len(data[0]) if rows else 0
    grid = GridspecLayout(rows, cols)
    for i in range(rows):
        for j in range(cols):
            label_value = str(data[j][i])
            if label_value == "food":
                label_value = "🍎"
            if label_value == "player":
                label_value = "🐍"
            grid[i, j] = Label(value=str(label_value), layout={'border': '1px solid black', 'width': '25px', 'height': '25px', 'padding': '0px', 'margin': '0px', 'display': 'flex', 'justify_content': 'center', 'align_items': 'center'})
    return grid

#更新場地的函數
def update_grid(grid, data):
    rows = len(data)
    cols = len(data[0]) if rows else 0
    for i in range(rows):
        for j in range(cols):
            label_value = str(data[j][i])
            if label_value == "food":
                label_value = "🍎"
            if label_value == "player":
                label_value = "🐍"
            grid[i, j].value = label_value

# 產生初始地圖
map_widget = create_grid(game.map)

# 設定控制鍵
up_button = Button(description="Up")
down_button = Button(description="Down")
left_button = Button(description="Left")
right_button = Button(description="Right")
start_button = Button(description="Start")

# 開始遊戲函數
def start_game(b):
    global game_active
    if game_active:
        return
    
    game_active = True
    print("Game started!")
    game.spawn_snake("player", 5)
    global start_time
    start_time = time.time()  
    asyncio.create_task(game_loop())

# 設定按鍵
up_button.on_click(lambda b: game.change_direction("player",(0, -1)))
down_button.on_click(lambda b: game.change_direction("player",(0, 1)))
left_button.on_click(lambda b: game.change_direction("player",(-1, 0)))
right_button.on_click(lambda b: game.change_direction("player",(1, 0)))
start_button.on_click(start_game)

# 設定遊戲迴圈
async def game_loop():
    global game_active, start_time
    while game_active:
        elapsed_time = time.time() - start_time
        if len(game.snakes) == 0:
            print("Game over!")
            break
        if elapsed_time >= FRAME_RATE:
            game.update()
            update_grid(map_widget, game.map)
            start_time = time.time()
        await asyncio.sleep(FRAME_RATE)


# 產生輸入方塊
keyboard_input = Text(layout={'width': '100px'})

# 輸入方向
def on_text_change(change):
    value = change['new']
    if value == 'w':
        game.change_direction("player", (0, -1))
    elif value == 's':
        game.change_direction("player", (0, 1))
    elif value == 'a':
        game.change_direction("player", (-1, 0))
    elif value == 'd':
        game.change_direction("player", (1, 0))
    elif value == ' ':
        start_game(None)
    keyboard_input.value = ''

#繪製遊戲畫面
keyboard_input.observe(on_text_change, 'value')
instruction_label_start = Label(value="（記得先輸入「空白鍵」來開始！）")
instruction_label = Label(value="輸入你想移動的方向 (w/a/s/d):")
keyboard = VBox([instruction_label, keyboard_input, instruction_label_start])

vbox_layout = Layout(width='auto', height='500px', justify_content='center', align_items='center', display='flex')
buttons = VBox([up_button, down_button, left_button, right_button])
game_display = HBox([map_widget, keyboard], layout=vbox_layout)

display(game_display)

Game started!
Changing direction of player to (0, -1)
Changing direction of player to (1, 0)
Changing direction of player to (0, 1)
Changing direction of player to (-1, 0)
Changing direction of player to (0, 1)
Changing direction of player to (-1, 0)
Changing direction of player to (0, 1)
Changing direction of player to (-1, 0)
Changing direction of player to (0, 1)
Changing direction of player to (-1, 0)
Changing direction of player to (0, -1)
Changing direction of player to (1, 0)
Changing direction of player to (0, 1)
Changing direction of player to (1, 0)
Changing direction of player to (0, 1)
Game over!
